In [20]:
import json
from collections import defaultdict

import pandas as pd
import tqdm

In [21]:
DATA_DIR = "../botify/data/"

track_with_recs = pd.read_json(f"{DATA_DIR}recommendations_contextual.json", lines=True)

In [22]:
track_by_id = {track["track"]: track for _, track in track_with_recs.iterrows()}

In [23]:
k = 40
max_tracks_from_same_artist = 10

with open(DATA_DIR + f"recommendations_{k}_{max_tracks_from_same_artist}.json", "w") as rf:
    for _, track in tqdm.tqdm(track_with_recs.iterrows()):
        track_index = track["track"]
        recommendations = track["recommendations"]

        filtered_recommendations = []

        artists = defaultdict(int)
        for neighbour in recommendations:
            recommended_track = track_by_id[neighbour]
            recommended_track_index = int(recommended_track["track"])

            if recommended_track_index == track_index:
                continue

            artist = recommended_track["artist"]
            if artists[artist] >= max_tracks_from_same_artist:
                continue
            artists[artist] += 1

            filtered_recommendations.append(recommended_track_index)

            if len(filtered_recommendations) == k:
                break

        track_with_recommendations = dict(track)
        track_with_recommendations["recommendations"] = filtered_recommendations

        rf.write(json.dumps(track_with_recommendations) + "\n")

50000it [00:16, 3020.83it/s]
